In [12]:
import pandas as pd

URL = 'https://en.wikipedia.org/wiki/NBA_All-Star_Game'
df = pd.read_html(URL)[2]
df.head(5)

,Year,Result,Host arena,Host city,Game MVP
0,1951,"East 111, West 94",Boston Garden,"Boston, Massachusetts","Ed Macauley, Boston Celtics"
1,1952,"East 108, West 91",Boston Garden (2),"Boston, Massachusetts (2)","Paul Arizin, Philadelphia Warriors"
2,1953,"West 79, East 75",Allen County War Memorial Coliseum,"Fort Wayne, Indiana","George Mikan, Minneapolis Lakers"
3,1954,"East 98, West 93 (OT)",Madison Square Garden III**,"New York City, New York","Bob Cousy, Boston Celtics"
4,1955,"East 100, West 91",Madison Square Garden III** (2),"New York City, New York (2)","Bill Sharman, Boston Celtics"


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year        74 non-null     object
 1   Result      74 non-null     object
 2   Host arena  74 non-null     object
 3   Host city   74 non-null     object
 4   Game MVP    72 non-null     object
dtypes: object(5)
memory usage: 3.0+ KB


The data you read in does not contain columns that store the information you need. You’ll need to remove columns that we don’t care about and make new columns with information we do care about. All this is done on a row-by-row basis, thus keeping all the original rows. The outcome is code that prints a DataFrame containing three columns: Eastern Conference score, Western Conference score, and the host city. The column index is the names of the columns. The row index is the year.

1. Remove the “Game MVP” column from the DataFrame.

2. Remove the “Host arena” column from the DataFrame.

3. Change the “Host city” column entries so that only the city name appears, and nothing after it. In other words, remove the state and the number of times the city hosted the game, if they were included.

4. Add two more columns, effectively splitting the “Results” column in two: (1) one column will contain integers in every row, representing the score for the Eastern Conference, and (2) the other column will contain integers in every row, representing the score for the Western Conference.

In [14]:
drop_list = ['Host arena', 'Game MVP']

for d in drop_list:
  df.drop(d, axis = 1, inplace = True)

df.head(3)

,Year,Result,Host city
0,1951,"East 111, West 94","Boston, Massachusetts"
1,1952,"East 108, West 91","Boston, Massachusetts (2)"
2,1953,"West 79, East 75","Fort Wayne, Indiana"


In [15]:
df.index = df['Year']
df.drop('Year', axis = 1, inplace = True)

df['Result'] = df['Result'].str.replace(',','')

def east (text):
  txt_list = text.split()
  v = str()
  
  try:
    v = int(txt_list[txt_list.index('East') + 1])
  except:
    pass

  return v

def west (text):
  txt_list = text.split()
  v = str()
  
  try:
    v = int(txt_list[txt_list.index('West') + 1])

  except:
    pass

  return v


df.insert(loc = 0, column = 'East', value = df['Result'].apply(east))
df.insert(1, 'West', df['Result'].apply(west))

In [16]:
def separator(text):
  L = text.split(',')
  return L[0]

df['Host city'] = df['Host city'].apply(separator)

In [17]:
df.drop('Result', 1, inplace = True)
df = df[df['East'] != '']
df.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,East,West,Host city
Year,,,
1951,111,94,Boston
1952,108,91,Boston
1953,75,79,Fort Wayne
1954,98,93,New York City
1955,100,91,New York City


Report counts for the score differences. Determine which final score difference between the two teams occurred most often. The end outcome is code that prints a DataFrame whose index is the difference, containing one column: the count of the difference.

1. Create a new column containing the difference between the Eastern and Western Conference teams.

2. Group the DataFrame by the new difference column. Make a new DataFrame that only contains the difference as the index and the count as the only column.

3. Sort the DataFrame so that you can quickly find the maximum value for the difference.

4. Print the sorted DataFrame.

In [18]:
dif_df = pd.DataFrame({'dif': abs(df['East'] - df['West'])})
dif_df

,dif
Year,
1951,17
1952,17
1953,4
1954,5
1955,9
...,...
2013,5
2014,8
2015,5


In [19]:
#Ahora queremos agrupar con Groupby para contar el número de puntos de diferencias
dif_gby = dif_df.groupby('dif')
count = dif_gby.size() #Cuenta cuántos
count = count.sort_values(ascending = False)
count.head(5)

dif
5     7
9     5
2     5
11    5
1     4
dtype: int64

You’ll group the data by how many times cities hosted an NBA All-Star Game. If any cities hosted more than once, then you’ll average all values for all those cities in each column. Then, you’ll only choose rows where the cities hosted the game more than once. The outcome is code that prints a DataFrame with the host city as the row index. It contains three columns: Eastern Conference average score across the same city, Western Conference score across the same city, and the count for how many times that city hosted the game.

1.   Group the DataFrame by the host city. Use the mean as the function to apply on columns where there is more than one value to group together. The index should now be the host city instead of the year.

2. Add the count of the host city as a column to your DataFrame.

3. Sort the DataFrame on the count column.

4. Only keep rows where the count column is greater than 1.

5. Print the DataFrame.

In [20]:
fil_df = df[['Host city', 'East', 'West']]

In [31]:
#Mean of each group
city_gby = fil_df.groupby('Host city')
A = pd.DataFrame(city_gby.mean().round(2).sort_values('East',ascending = False))

#How many NBA All-Star Game have token place in each city?
B = pd.DataFrame(city_gby.size())

#And we merge both
final_df = pd.merge(A, B, left_index=True, right_index=True)

final_df.head(6)

,East,West,0
Host city,,,
Toronto,173.00,196.00,1
New Orleans,159.67,158.33,3
Landover,144.00,136.00,1
Arlington,141.00,139.00,1
Denver,139.50,130.00,2
Atlanta,139.00,140.00,2


In [33]:
#Keep the 'Host cities' greater than 1
final_df = final_df[final_df[0] > 1].sort_values(0, ascending= True)
final_df

,East,West,0
Host city,,,
Denver,139.50,130.00,2
Atlanta,139.00,140.00,2
Seattle,136.00,144.00,2
Orlando,131.00,152.50,2
Inglewood,121.00,117.50,2
Chicago,121.00,108.50,2
New Orleans,159.67,158.33,3
Houston,131.33,135.33,3
Los Angeles,130.00,130.67,3
